In [28]:
import underthesea
import re
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import naive_bayes
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
import joblib


In [2]:
def lower_tokenize(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    tokens = underthesea.word_tokenize(text)
    # print("Lower",tokens)
    return tokens

In [3]:
def remove_stopword(tokens):
    with open("vietnamese-stopwords.txt","r",encoding="utf-8") as file:
        stopwords = file.readlines()
    stopwords = [word.strip() for word in stopwords]
    final_tokens = [token for token in tokens if token not in stopwords]
    final_text = ' '.join(final_tokens)
    # print("Stopword",final_text)
    return final_text


In [4]:
df = pd.read_csv('../Crawl/Data.csv')

In [5]:
# df = df.head(1000)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  25000 non-null  int64 
 1   Danh mục    25000 non-null  object
 2   Nội dung    25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [7]:
df['Danh mục'].value_counts()

Danh mục
Xe                  3077
Giới trẻ            3001
Giáo dục            2602
Sức khỏe            2573
Thời sự             2449
Thế giới            2309
Văn hóa             2307
Công nghệ - Game    2142
Thể thao            1364
Đời sống            1197
Kinh tế             1098
Giải trí             780
Du lịch              101
Name: count, dtype: int64

In [8]:
df = df.dropna()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  25000 non-null  int64 
 1   Danh mục    25000 non-null  object
 2   Nội dung    25000 non-null  object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


In [9]:
content=df['Nội dung'].apply(lambda x: lower_tokenize(x))
content=content.apply(lambda x: remove_stopword(x))
label=df['Danh mục']

In [10]:
content

0        gói bao gồm chương trình tài chính sáng kiến n...
1        cơ quan an ninh điều tra a bộ công an phát đi ...
2        tand tphcm kháng cáo nhẹ hình phạt cựu giám đố...
3        chương trình phiên họp chất vấn trả lời chất v...
4        tand tỉnh vĩnh long phiên tòa hình sự sơ thẩm ...
                               ...                        
24995    hành trình tỏa rực rỡ khởi nghiệp song hoa điế...
24996    pascal đề cử emmy vai diễn chính phim the last...
24997    phim trấn thành dẫn đầu phòng vé quafbnvtheo g...
24998    phát hành link ca khúc hoàng thùy linh nhận mư...
24999    tết khán giả lê giang phim bầuđpcctối diễn viê...
Name: Nội dung, Length: 25000, dtype: object

In [11]:
label

0        Thế giới
1         Thời sự
2         Thời sự
3         Thời sự
4         Thời sự
           ...   
24995    Giải trí
24996     Văn hóa
24997    Giải trí
24998    Giải trí
24999    Giải trí
Name: Danh mục, Length: 25000, dtype: object

In [42]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(content, label, test_size=0.3, random_state=10)
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)
# Mã hóa nhãn mục tiêu
encoder = preprocessing.LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [37]:
X_train_tfidf.shape

(17500, 5000)

In [38]:
X_test_tfidf.shape

(7500, 5000)

In [39]:
#Huấn luyện mô hình Naive Bayes
MultiNB = naive_bayes.MultinomialNB(alpha=0.2)
# fit the training dataset on the classifier
MultiNB.fit(X_train_tfidf, y_train)
# Dự đoán nhãn
predictions_nb = MultiNB.predict(X_test_tfidf)

In [40]:
# Accuracy
accuracy = accuracy_score(predictions_nb, y_test)
print(f"Đánh giá bằng: Accuracy: {accuracy}")

# Precision
precision = precision_score(predictions_nb, y_test, average='weighted')
print(f"Đánh giá bằng: Precision: {precision}")

# Recall
recall = recall_score(predictions_nb, y_test, average='weighted')
print(f"Đánh giá bằng: Recall: {recall}")

# F1 Score
f1 = f1_score(predictions_nb, y_test, average='weighted')
print(f"Đánh giá bằng: F1 Score: {f1}")

Đánh giá bằng: Accuracy: 0.8196
Đánh giá bằng: Precision: 0.8198544687794502
Đánh giá bằng: Recall: 0.8196
Đánh giá bằng: F1 Score: 0.8180657859781402


In [41]:
# Chuẩn bị dữ liệu kiểm thử mới
new_data = ["Tôi thích học Toán",
            "VIệt Nam ra sân thắng Lào với tỉ số 10-0",
            "Bạo lực học đường là vấn đề được quan tâm nhất hiện nay",
            "Xe Ariblade 165CC có ngoại hình siêu đẹp"]

# Tiền xử lý dữ liệu kiểm thử mới
new_data= [lower_tokenize(text) for text in new_data]
new_data = [remove_stopword(tokens) for tokens in new_data] # Loại bỏ stop words
# Chuyển đổi dữ liệu kiểm thử mới thành đặc trưng TF-IDF
new_data_tfidf = tfidf_vectorizer.transform(new_data)

# Dự đoán nhãn với dữ liệu kiểm thử mới
new_predictions = MultiNB.predict(new_data_tfidf)

# Chuyển đổi nhãn được dự đoán về dạng ban đầu
new_predictions_labels = encoder.inverse_transform(new_predictions)
# In ra các dự đoán
for i, text in enumerate(new_data):
    print(f"Nội dung: ",text)
    print(f"Dự đoán danh mục: ",new_predictions_labels[i])
    print("-" * 40)

Nội dung:  học toán
Dự đoán danh mục:  Giáo dục
----------------------------------------
Nội dung:  việt nam sân thắng lào tỉ số
Dự đoán danh mục:  Thể thao
----------------------------------------
Nội dung:  bạo lực học đường
Dự đoán danh mục:  Giáo dục
----------------------------------------
Nội dung:  xe ariblade cc ngoại hình siêu đẹp
Dự đoán danh mục:  Xe
----------------------------------------


In [18]:
joblib.dump((tfidf_vectorizer,MultiNB,encoder), "model.joblib")

['model.joblib']